# NYC Parking Tickets: An Exploratory Analysis

In [1]:
from pyspark.sql import SparkSession
# setting up the spark session
spark = SparkSession \
    .builder \
    .appName("NYC Parking Tickets EDA") \
    .getOrCreate()

In [2]:
# reading the data
nycparking_df = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("/common_folder/nyc_parking/Parking_Violations_Issued_-_Fiscal_Year_2017.csv")\
  .coalesce(3) 


In [3]:
# checking how the data looks like
nycparking_df.show(10)

+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|Summons Number|Plate ID|Registration State|         Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|    5092469481| GZH7067|                NY|2016-07-10 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0143A|
|    5092451658| GZH7067|                NY|2016-07-08 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0400P|
|    4006265037| FZX9232|                NY|2016-08-23 00:00:00|             5|             SUBN|        FORD|                 0|              0|         0233P|
|    8478629828| 66623ME|         

## Assumptions

Before moving ahead let's list down the set of assumptions for the below analysis, 
1. `Issue Date` defines which year a certain datapoint belongs to.
2. All the `Registration State` except `99`(since given in the problem statement) are correct values.
3. All the values of `Plate ID`, `Violation Code`, `Vehicle Body Type`, `Vehicle Make` and `Violation Time` are correct(except NULLs).
4. All the values for `Violation Precinct` and `Issuer Precinct` are correct(except 0 and NULL).
5. Hints given with the problem statements are just for the sake of help and are not mandatory to be followed.
6. Changing the format of a column for bettwr usability for analysis is not prohibited(unless stated otherwise in the problem statement).

## Examine the data

Let's first check the basic sanity of the data we are expecting to work with, to start with the very first thing is the year this data belongs to has to be year `2017` as per the problem statement.

In [4]:
# checking the data duration using the Ticket "Issue Date"
from pyspark.sql.functions import year
nycparking_df.where(year('Issue Date') != 2017).count()

5371110

So we have `5371110` records in the dataset which doesn't actually belong to year `2017` and therefore should be removed from the data under consideration.

In [5]:
# creating a new dataframe for the records from year 2017 only
nycpark2017_df = nycparking_df.where(year('Issue Date') == 2017)
nycpark2017_df.show(10)

+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|Summons Number|Plate ID|Registration State|         Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|    8478629828| 66623ME|                NY|2017-06-14 00:00:00|            47|             REFG|       MITSU|                14|             14|         1120A|
|    5096917368| FZD8593|                NY|2017-06-13 00:00:00|             7|             SUBN|       ME/BE|                 0|              0|         0852P|
|    1407740258| 2513JMG|                NY|2017-01-11 00:00:00|            78|             DELV|       FRUEH|               106|            106|         0015A|
|    1413656420|T672371C|         


### 1. Find the total number of tickets for the year.

In [6]:
# counting the total tickets for the entire year
nycpark2017_df.select('Summons Number').distinct().count()

5431918

Therefore, there are `5431918` tickets issued during year 2017.

### 2. Find out the number of unique states from where the cars that got parking tickets came. (Hint: Use the column 'Registration State'.)   There is a numeric entry '99' in the column, which should be corrected. Replace it with the state having the maximum entries. Provide the number of unique states again.

In [7]:
# counting the total unique states
nycpark2017_df.select('Registration State').distinct().count()

65

In [8]:
# checking the unique states names
nycpark2017_df.select('Registration State').distinct().show(65)

+------------------+
|Registration State|
+------------------+
|                SC|
|                AZ|
|                NS|
|                LA|
|                MN|
|                NJ|
|                DC|
|                OR|
|                99|
|                VA|
|                RI|
|                KY|
|                WY|
|                BC|
|                NH|
|                MI|
|                GV|
|                NV|
|                QB|
|                WI|
|                ID|
|                CA|
|                CT|
|                NE|
|                MT|
|                NC|
|                VT|
|                MD|
|                DE|
|                MO|
|                IL|
|                ME|
|                MB|
|                WA|
|                ND|
|                MS|
|                AL|
|                IN|
|                OH|
|                TN|
|                NM|
|                IA|
|                PA|
|                SD|
|            

As suggested in the problem statement itself, there is only one odd entry `99`, let's impute this with the mode of `Registration State`. 

In [9]:
# finding the mode of column Registration State
from pyspark.sql.functions import count, desc
nycpark2017_df.groupBy('Registration State').agg(count('Summons Number').alias('Frequency')).orderBy(desc('Frequency')).show()

+------------------+---------+
|Registration State|Frequency|
+------------------+---------+
|                NY|  4273951|
|                NJ|   475825|
|                PA|   140286|
|                CT|    70403|
|                FL|    69468|
|                IN|    45525|
|                MA|    38941|
|                VA|    34367|
|                MD|    30213|
|                NC|    27152|
|                TX|    18827|
|                IL|    18666|
|                GA|    17537|
|                99|    16055|
|                AZ|    12379|
|                OH|    12281|
|                CA|    12153|
|                ME|    10806|
|                SC|    10395|
|                MN|    10083|
+------------------+---------+
only showing top 20 rows



So the Mode is `NY`, which is to be imputed in the dataframe. 

In [10]:
# imputing the wrong Registration State '99' with the Mode 'NY'
from pyspark.sql.functions import when
nycpark2017_df = nycpark2017_df.withColumn('Registration State', \
                                           when(nycpark2017_df['Registration State']=='99', 'NY')\
                                           .otherwise(nycpark2017_df['Registration State']))

In [11]:
nycpark2017_df.select('Registration State').distinct().count()

64

Therefore, there are **`64` unique states** from where the cars that got parking tickets came.

Let's next check the NULL values in the dataframe if exists. 

In [12]:
from pyspark.sql.functions import isnan, col
nycpark2017_df.select([count(when(col(c).isNull(), c)).alias(c) for c in nycpark2017_df.columns]).show()

+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+---------------+--------------+
|Summons Number|Plate ID|Registration State|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+---------------+--------------+
|             0|       0|                 0|         0|             0|                0|           0|                 0|              0|             0|
+--------------+--------+------------------+----------+--------------+-----------------+------------+------------------+---------------+--------------+



So there are no NULL values in our dataset, further to check the Columns for any errorneous values, two columns are susceptible to errorneous data `Vehicle Body Type` and `Vehicle Make`. 

In [13]:
# checking the unique vehicle body types
nycpark2017_df.select('Vehicle Body Type').distinct().show(50)

+-----------------+
|Vehicle Body Type|
+-----------------+
|              BOX|
|             ARMO|
|               3P|
|             COOM|
|             GARB|
|             TRUV|
|                K|
|             ELEC|
|             FRET|
|              MDX|
|             BOAT|
|              TRK|
|             4DSW|
|             VAND|
|              IML|
|             FREG|
|             WINN|
|             MOTC|
|             MOTI|
|               RG|
|              UTI|
|               RF|
|             DUMD|
|              INE|
|               TC|
|               PC|
|             ORPP|
|             TNRC|
|             KEVG|
|             STAK|
|             ACCO|
|             ILQT|
|               SC|
|              RED|
|             DUIM|
|             BUSS|
|             INNI|
|              SNA|
|               IC|
|               PU|
|             TAHO|
|             TRLC|
|              BLK|
|              TRE|
|             ONGO|
|               NS|
|             REFY|


In [14]:
# checking the unique vehicle body types
nycpark2017_df.select('Vehicle Make').distinct().show(50)

+------------+
|Vehicle Make|
+------------+
|       ENFIE|
|           K|
|       ARUCA|
|       INTIL|
|        FRER|
|        YRAN|
|        FHHT|
|       N. CA|
|         LEN|
|        VACO|
|          MM|
|        FRET|
|         HYL|
|       SALEE|
|       INFTL|
|         BOX|
|       HTYUN|
|        FIER|
|         OWL|
|        ANNA|
|       LA//R|
|       PETER|
|       VERMA|
|        WINN|
|        FREG|
|       UTILL|
|         TRK|
|         KAT|
|        L/RO|
|       ROUGH|
|       FORDB|
|       SPRNT|
|        LIML|
|        WE/C|
|       HINOQ|
|       INETR|
|       HYUN1|
|       UTILT|
|        SABR|
|       TOW M|
|       BOUND|
|        SOLE|
|       FLIDE|
|        PIGG|
|         IGC|
|         INL|
|        VOWA|
|       YAMAH|
|       POLAR|
|       HYADA|
+------------+
only showing top 50 rows



In both the columns we can see a number of suspicious entries which are very likely to be wrong, like `Vehicle Body Type` (`K`, `4W0`) and `Vehicle Make` (`LA//R`, `N. CA`); but since we do not have enough data to correct these values, we will assume these values exist.

Now before moving to the next section, let's create a view out of our final dataframe `nycpark2017_df`. 

In [15]:
# persisting data into Memory and disk and making a view out of it
import pyspark
nycpark2017_df.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
nycpark2017_df.createOrReplaceTempView("nycparktbl")

## Aggregation tasks

### 1. How often does each violation code occur? Display the frequency of the top five violation codes.

In [16]:
# counting the frequency of each Violation Code in the dataset
nycpark2017_df.groupBy('Violation Code').agg(count('Summons Number').alias('Frequency')).orderBy(desc('Frequency')).show(5)

+--------------+---------+
|Violation Code|Frequency|
+--------------+---------+
|            21|   768087|
|            36|   662765|
|            38|   542079|
|            14|   476664|
|            20|   319646|
+--------------+---------+
only showing top 5 rows



i.e, **Violation Code `21`** is the most common of all. 

### 2. How often does each 'vehicle body type' get a parking ticket? How about the 'vehicle make'? (Hint: Find the top 5 for both.)

In [17]:
# checking the frequency of vehicle body type
nycpark2017_df.groupBy('Vehicle Body Type').agg(count('Summons Number').alias('Frequency')).orderBy(desc('Frequency')).show(5)

+-----------------+---------+
|Vehicle Body Type|Frequency|
+-----------------+---------+
|             SUBN|  1883954|
|             4DSD|  1547312|
|              VAN|   724029|
|             DELV|   358984|
|              SDN|   194197|
+-----------------+---------+
only showing top 5 rows



In [18]:
# checking the frequency of vehicle make
nycpark2017_df.groupBy('Vehicle Make').agg(count('Summons Number').alias('Frequency')).orderBy(desc('Frequency')).show(5)

+------------+---------+
|Vehicle Make|Frequency|
+------------+---------+
|        FORD|   636844|
|       TOYOT|   605291|
|       HONDA|   538884|
|       NISSA|   462017|
|       CHEVR|   356032|
+------------+---------+
only showing top 5 rows



As it could be clearly seen from the above results, **Vehicles with body type `SUBN`** are the major receivers of a parking ticket, however if analysed based on the **Vehicle Make `FORD`** owners have the maximum contribution to the population.

### 3. A precinct is a police station that has a certain zone of the city under its command. Find the (5 highest) frequencies of tickets for each of the following:

#### 3.1. 'Violation Precinct' (This is the precinct of the zone where the violation occurred). Using this, can you draw any insights for parking violations in any specific areas of the city?

In [19]:
# checking the violation frequency based on Violation Precinct
nycpark2017_df.groupBy('Violation Precinct').agg(count('Summons Number').alias('Frequency')).orderBy(desc('Frequency')).show(6)

+------------------+---------+
|Violation Precinct|Frequency|
+------------------+---------+
|                 0|   925596|
|                19|   274445|
|                14|   203553|
|                 1|   174702|
|                18|   169131|
|               114|   147444|
+------------------+---------+
only showing top 6 rows



**Precinct `19` and `14` are the top two** candidates with the highest parking violations and therefore falls in the **most violation prone zone(s) of the city**.

#### 3.2. 'Issuer Precinct' (This is the precinct that issued the ticket.) 
#### Here, you would have noticed that the dataframe has the'Violating Precinct' or 'Issuing Precinct' as '0'. These are erroneous entries. Hence, you need to provide the records for five correct precincts. (Hint: Print the top six entries after sorting.)

In [20]:
# checking the violation frequency based on Issuer Precinct
nycpark2017_df.groupBy('Issuer Precinct').agg(count('Summons Number').alias('Frequency')).orderBy(desc('Frequency')).show(6)

+---------------+---------+
|Issuer Precinct|Frequency|
+---------------+---------+
|              0|  1078406|
|             19|   266961|
|             14|   200495|
|              1|   168740|
|             18|   162994|
|            114|   144054|
+---------------+---------+
only showing top 6 rows



Clearly, **Precinct `19` is the highest issuer of the tickets** followed by precinct `14` for the very obvious reason as a conclusion from our observation above.

### 4. Find the violation code frequencies for three precincts that have issued the most number of tickets. Do these precinct zones have an exceptionally high frequency of certain violation codes? Are these codes common across precincts? 

In [21]:
# violation code frequency for Issuer precinct 19
spark.sql("select `Violation Code`, count(*) as Frequency from \
(select * from nycparktbl where `Issuer Precinct` == 19) group by `Violation Code` order by Frequency desc").show()

+--------------+---------+
|Violation Code|Frequency|
+--------------+---------+
|            46|    48445|
|            38|    36386|
|            37|    36056|
|            14|    29797|
|            21|    28415|
|            20|    14629|
|            40|    11416|
|            16|     9926|
|            71|     7493|
|            19|     6856|
|            10|     5643|
|            84|     4910|
|            70|     4459|
|            18|     3148|
|            69|     2910|
|            31|     2080|
|            53|     1736|
|            50|     1483|
|            17|     1464|
|            48|     1460|
+--------------+---------+
only showing top 20 rows



As we can see `46`, `38`, `37`, `14`, `21` are the Violation codes which are extremely frequently charged by Precinct 19.

In [22]:
# violation code frequency for Issuer precinct 14
spark.sql("select `Violation Code`, count(*) as Frequency from \
(select * from nycparktbl where `Issuer Precinct` == 14) group by `Violation Code` order by Frequency desc").show()

+--------------+---------+
|Violation Code|Frequency|
+--------------+---------+
|            14|    45036|
|            69|    30464|
|            31|    22555|
|            47|    18364|
|            42|    10027|
|            46|     7679|
|            19|     7031|
|            84|     6743|
|            82|     5052|
|            40|     3582|
|            17|     3534|
|            38|     3269|
|             9|     2874|
|            20|     2761|
|            71|     2757|
|            13|     2701|
|            48|     2439|
|            89|     1960|
|            50|     1824|
|            11|     1745|
+--------------+---------+
only showing top 20 rows



Precint 14 has charged for Violation codes `14`, `69` and `31` heavily.

In [23]:
# violation code frequency for Issuer precinct 1
spark.sql("select `Violation Code`, count(*) as Frequency from \
(select * from nycparktbl where `Issuer Precinct` == 1) group by `Violation Code` order by Frequency desc").show()

+--------------+---------+
|Violation Code|Frequency|
+--------------+---------+
|            14|    38354|
|            16|    19081|
|            20|    15408|
|            46|    12745|
|            38|     8535|
|            17|     7526|
|            37|     6470|
|            31|     5853|
|            69|     5672|
|            19|     5375|
|            10|     4712|
|            40|     4592|
|            21|     4055|
|            71|     3581|
|            84|     3310|
|            42|     2708|
|            51|     2223|
|             9|     2206|
|            70|     2183|
|            48|     1907|
+--------------+---------+
only showing top 20 rows



In case of precinct 1, the most commonly occuring Violation codes are `14`, `16` and `20`. 

Now, comparing the results from  the top three Violation ticket issueing precincts it is crystal clear that **Violation code `14` is most likely to happen**.

### 5. Find out the properties of parking violations across different times of the day:

#### > Find a way to deal with missing values, if any. (Hint: Check for the null values using 'isNull' under the SQL. Also, to remove the null values, check the 'dropna' command in the API documentation.)

In [24]:
spark.sql("select count(*) from nycparktbl where `Violation Time` is null").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



As we had checked this already, there are no NULL values in the column `Violation Time`.

#### > The Violation Time field is specified in a strange format. Find a way to make this a time attribute that you can use to divide into groups.

The best way to achieving this is converting the column `Violation Time` into the 24 hrs clock i.e. from format HHMMA/P to HHMM.

In [25]:
from pyspark.sql.functions import concat, lit
# converting the cells for 12 AM to 00 and remove 'A', rest remains same
nycpark2017_df = nycpark2017_df.withColumn('Violation Time', \
                          when(((nycpark2017_df['Violation Time'].substr(1, 2)==12) & (nycpark2017_df['Violation Time'].substr(5,1)=='A')), concat(lit("00"), nycpark2017_df['Violation Time'].substr(3, 2)))\
                          .otherwise(nycpark2017_df['Violation Time']))
nycpark2017_df.show(10)

+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|Summons Number|Plate ID|Registration State|         Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|    8478629828| 66623ME|                NY|2017-06-14 00:00:00|            47|             REFG|       MITSU|                14|             14|         1120A|
|    5096917368| FZD8593|                NY|2017-06-13 00:00:00|             7|             SUBN|       ME/BE|                 0|              0|         0852P|
|    1407740258| 2513JMG|                NY|2017-01-11 00:00:00|            78|             DELV|       FRUEH|               106|            106|         0015A|
|    1413656420|T672371C|         

In [26]:
# converting the remaining cells for 12 PM to remove 'P', rest remains same
nycpark2017_df = nycpark2017_df.withColumn('Violation Time', \
                          when(((nycpark2017_df['Violation Time'].substr(1, 2)==12) & (nycpark2017_df['Violation Time'].substr(5,1)=='P')), nycpark2017_df['Violation Time'].substr(1, 4))\
                          .otherwise(nycpark2017_df['Violation Time']))
nycpark2017_df.show(20)

+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|Summons Number|Plate ID|Registration State|         Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|    8478629828| 66623ME|                NY|2017-06-14 00:00:00|            47|             REFG|       MITSU|                14|             14|         1120A|
|    5096917368| FZD8593|                NY|2017-06-13 00:00:00|             7|             SUBN|       ME/BE|                 0|              0|         0852P|
|    1407740258| 2513JMG|                NY|2017-01-11 00:00:00|            78|             DELV|       FRUEH|               106|            106|         0015A|
|    1413656420|T672371C|         

In [27]:
# converting the remaining cells with 'P' to add 12hrs, rest remains same
nycpark2017_df = nycpark2017_df.withColumn('Violation Time', \
                          when((nycpark2017_df['Violation Time'].substr(5,1)=='P'), concat((nycpark2017_df['Violation Time'].substr(1, 2) + 12).cast("int"), nycpark2017_df['Violation Time'].substr(3, 2)))\
                          .otherwise(nycpark2017_df['Violation Time']))
nycpark2017_df.show(20)

+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|Summons Number|Plate ID|Registration State|         Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|    8478629828| 66623ME|                NY|2017-06-14 00:00:00|            47|             REFG|       MITSU|                14|             14|         1120A|
|    5096917368| FZD8593|                NY|2017-06-13 00:00:00|             7|             SUBN|       ME/BE|                 0|              0|          2052|
|    1407740258| 2513JMG|                NY|2017-01-11 00:00:00|            78|             DELV|       FRUEH|               106|            106|         0015A|
|    1413656420|T672371C|         

In [28]:
# converting the remaining cells with 'A' to remove 'A', rest remains same
nycpark2017_df = nycpark2017_df.withColumn('Violation Time', \
                          when((nycpark2017_df['Violation Time'].substr(5,1)=='A'), nycpark2017_df['Violation Time'].substr(1, 4))\
                          .otherwise(nycpark2017_df['Violation Time']))
nycpark2017_df.show(20)

+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|Summons Number|Plate ID|Registration State|         Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|    8478629828| 66623ME|                NY|2017-06-14 00:00:00|            47|             REFG|       MITSU|                14|             14|          1120|
|    5096917368| FZD8593|                NY|2017-06-13 00:00:00|             7|             SUBN|       ME/BE|                 0|              0|          2052|
|    1407740258| 2513JMG|                NY|2017-01-11 00:00:00|            78|             DELV|       FRUEH|               106|            106|          0015|
|    1413656420|T672371C|         

Which brings us to the state where we can very easily play with this column.

#### > Divide 24 hours into six equal discrete bins of time. Choose the intervals as you see fit. For each of these groups, find the three most commonly occurring violations. 

To divide the 24 hrs into 6 equal bins, we will define these as [00-04), [04-08), [08-12), [12-16), [16-20) and [20-00).

In [29]:
# Finding the top 3 Violation Codes for Bin1 [00-04)
spark.sql("select `Violation Code`, count(*) as Frequency from \
            (select `Summons Number`, `Violation Code` from nycparktbl where substr(`Violation Time`, 1, 2) in ('00', '01', '02', '03')) \
            group by `Violation Code` order by Frequency desc").show(3)

+--------------+---------+
|Violation Code|Frequency|
+--------------+---------+
|            38|   184993|
|            36|   184293|
|            37|   130718|
+--------------+---------+
only showing top 3 rows



In [30]:
# Finding the top 3 Violation Codes for Bin2 [04-08)
spark.sql("select `Violation Code`, count(*) as Frequency from \
            (select `Summons Number`, `Violation Code` from nycparktbl where substr(`Violation Time`, 1, 2) in ('04', '05', '06', '07')) \
            group by `Violation Code` order by Frequency desc").show(3)

+--------------+---------+
|Violation Code|Frequency|
+--------------+---------+
|            14|   150016|
|            38|   104128|
|            40|    86092|
+--------------+---------+
only showing top 3 rows



In [31]:
# Finding the top 3 Violation Codes for Bin3 [08-12)
spark.sql("select `Violation Code`, count(*) as Frequency from \
            (select `Summons Number`, `Violation Code` from nycparktbl where substr(`Violation Time`, 1, 2) in ('08', '09', '10', '11')) \
            group by `Violation Code` order by Frequency desc").show(3)

+--------------+---------+
|Violation Code|Frequency|
+--------------+---------+
|            21|   598254|
|            36|   348165|
|            38|   196917|
+--------------+---------+
only showing top 3 rows



In [32]:
# Finding the top 3 Violation Codes for Bin4 [12-16)
spark.sql("select `Violation Code`, count(*) as Frequency from \
            (select `Summons Number`, `Violation Code` from nycparktbl where substr(`Violation Time`, 1, 2) in ('12', '13', '14', '15')) \
            group by `Violation Code` order by Frequency desc").show(3)

+--------------+---------+
|Violation Code|Frequency|
+--------------+---------+
|            36|   101991|
|            21|    74615|
|            38|    56040|
+--------------+---------+
only showing top 3 rows



In [33]:
# Finding the top 3 Violation Codes for Bin5 [16-20)
spark.sql("select `Violation Code`, count(*) as Frequency from \
            (select `Summons Number`, `Violation Code` from nycparktbl where substr(`Violation Time`, 1, 2) in ('16', '17', '18', '19')) \
            group by `Violation Code` order by Frequency desc").show(3)

+--------------+---------+
|Violation Code|Frequency|
+--------------+---------+
|            98|        2|
|            21|        2|
|            46|        1|
+--------------+---------+



In [34]:
# Finding the top 3 Violation Codes for Bin6 [20-00)
spark.sql("select `Violation Code`, count(*) as Frequency from \
            (select `Summons Number`, `Violation Code` from nycparktbl where substr(`Violation Time`, 1, 2) in ('20', '21', '22', '23')) \
            group by `Violation Code` order by Frequency desc").show(3)

+--------------+---------+
|Violation Code|Frequency|
+--------------+---------+
|            17|        1|
|            78|        1|
|            40|        1|
+--------------+---------+



Therefore, from the above observations some trends we can derive are, 

* Violation code `38` is the most common on any day and is much frequent during early morning hrs.
* Violation code `21` is mostly common during the general working hrs of the major population.
* Evening hours(1600 to 0000 hrs) is the quietest time of the day.
* Morning (0800-1200 hrs) is the most busy time of a day. 

#### > Now, try another direction. For the three most commonly occurring violation codes, find the most common time of the day (in terms of the bins from the previous part).

As we already know from point-1 above, the three most common Violation codes are 21, 36 and 38 chronologically. Let's explore further...

In [35]:
# creating a view with the binned data
nycpark2017_df.select('Summons Number', 'Violation Code', \
                          when((nycpark2017_df['Violation Time'].substr(1, 2) <= 3), "Bin1")\
                          .when((nycpark2017_df['Violation Time'].substr(1, 2) <= 7), "Bin2")\
                          .when((nycpark2017_df['Violation Time'].substr(1, 2) <= 11), "Bin3")\
                          .when((nycpark2017_df['Violation Time'].substr(1, 2) <= 15), "Bin4")\
                          .when((nycpark2017_df['Violation Time'].substr(1, 2) <= 19), "Bin5")\
                          .when((nycpark2017_df['Violation Time'].substr(1, 2) <= 23), "Bin6")\
                          .otherwise(nycpark2017_df['Violation Time']).alias('Violation Bin'))\
.createOrReplaceTempView("nycparktbl_binned")

In [36]:
# checking the most common time of the day for Violation code 21
spark.sql("select `Violation Bin`, count(*) as Frequency from \
            (select * from nycparktbl_binned where `Violation Code`==21) \
            group by `Violation Bin` order by Frequency desc ").show(1)

+-------------+---------+
|Violation Bin|Frequency|
+-------------+---------+
|         Bin3|   598070|
+-------------+---------+
only showing top 1 row



In [37]:
# checking the most common time of the day for Violation code 36
spark.sql("select `Violation Bin`, count(*) as Frequency from \
            (select * from nycparktbl_binned where `Violation Code`==36) \
            group by `Violation Bin` order by Frequency desc ").show(1)

+-------------+---------+
|Violation Bin|Frequency|
+-------------+---------+
|         Bin3|   348165|
+-------------+---------+
only showing top 1 row



In [38]:
# checking the most common time of the day for Violation code 38
spark.sql("select `Violation Bin`, count(*) as Frequency from \
            (select * from nycparktbl_binned where `Violation Code`==38) \
            group by `Violation Bin` order by Frequency desc ").show(1)

+-------------+---------+
|Violation Bin|Frequency|
+-------------+---------+
|         Bin4|   240721|
+-------------+---------+
only showing top 1 row



As we can clearly see, for the top two **Violation codes, `21` and `36` the most common time of occurence is 08AM to 12PM**, while the third highest **Violation code `38` occurs mostly between 12PM to 04PM**.

### 6. Let’s try and find some seasonality in this data:

#### > First, divide the year into a certain number of seasons, and find the frequencies of tickets for each season. (Hint: Use Issue Date to segregate into seasons.)

Now as per the standard weather conditions in the New York City there are four seasons, `Dec to Feb is Winter` Season, `Mar to May is Spring` season, `Jun to Aug is Summer` season and `Sep to Nov is the Fall` season. Let's divide our dataset accordingly. 

In [39]:
# creating a view based on the seasons
from pyspark.sql.functions import month
nycpark2017_df.select('Summons Number', 'Violation Code', \
                          when(((month('Issue Date')==12) | (month('Issue Date')==1) | (month('Issue Date')==2)), "Winter")\
                          .when(((month('Issue Date')==3) | (month('Issue Date')==4) | (month('Issue Date')==5)), "Spring")\
                          .when(((month('Issue Date')==6) | (month('Issue Date')==7) | (month('Issue Date')==8)), "Summer")\
                          .when(((month('Issue Date')==9) | (month('Issue Date')==10) | (month('Issue Date')==11)), "Fall")\
                          .alias('Season'))\
.createOrReplaceTempView("nycparktbl_seasons")

In [40]:
# finding the frequency of ticket for each season
spark.sql("select Season, count(*) as Frequency from nycparktbl_seasons group by Season order by Frequency desc").show()

+------+---------+
|Season|Frequency|
+------+---------+
|Spring|  2873383|
|Winter|  1704690|
|Summer|   852866|
|  Fall|      979|
+------+---------+



Which says that **`Spring` has the highest frequency** of defaults as compared to any other season, while **`Winter` season stands second** in the row with just 59% of the frequency of Spring season.

#### > Then, find the three most common violations for each of these seasons. (Hint: You can use an approach similar to the one mentioned in the hint for question 4.)

In [41]:
# checking the 3 most common Violations for the Spring season
spark.sql("select `Violation Code`, count(*) as Frequency from \
            (select * from nycparktbl_seasons where `Season`=='Spring') \
            group by `Violation Code` order by Frequency desc ").show(3)

+--------------+---------+
|Violation Code|Frequency|
+--------------+---------+
|            21|   402424|
|            36|   344834|
|            38|   271167|
+--------------+---------+
only showing top 3 rows



In [42]:
# checking the 3 most common Violations for the Winter season
spark.sql("select `Violation Code`, count(*) as Frequency from \
            (select * from nycparktbl_seasons where `Season`=='Winter') \
            group by `Violation Code` order by Frequency desc ").show(3)

+--------------+---------+
|Violation Code|Frequency|
+--------------+---------+
|            21|   238183|
|            36|   221268|
|            38|   187386|
+--------------+---------+
only showing top 3 rows



In [43]:
# checking the 3 most common Violations for the Summer season
spark.sql("select `Violation Code`, count(*) as Frequency from \
            (select * from nycparktbl_seasons where `Season`=='Summer') \
            group by `Violation Code` order by Frequency desc ").show(3)

+--------------+---------+
|Violation Code|Frequency|
+--------------+---------+
|            21|   127352|
|            36|    96663|
|            38|    83518|
+--------------+---------+
only showing top 3 rows



In [44]:
# checking the 3 most common Violations for the Fall season
spark.sql("select `Violation Code`, count(*) as Frequency from \
            (select * from nycparktbl_seasons where `Season`=='Fall') \
            group by `Violation Code` order by Frequency desc ").show(3)

+--------------+---------+
|Violation Code|Frequency|
+--------------+---------+
|            46|      231|
|            21|      128|
|            40|      116|
+--------------+---------+
only showing top 3 rows



It is quite surprising to find out that, 
* Violation code `21` is common throughout the year.
* Violation codes `21`, `36` and `38` are continuosly the three top Violations from December to August and in the same order. 
* Fall season is significantly quiet as compared to other three seasons of the year.
* Fall season have different kind of Violations (`46` and `40`) that other 3 seasons.

### 7. The fines collected from all the instances of parking violation constitute a source of revenue for the NYC Police Department. Let’s take an example of estimating this for the three most commonly occurring codes:

#### > Find the total occurrences of the three most common violation codes.

In [45]:
spark.sql("select `Violation Code`, count(*) as Frequency from nycparktbl \
            group by `Violation Code` \
            order by Frequency desc").show(3)

+--------------+---------+
|Violation Code|Frequency|
+--------------+---------+
|            21|   768087|
|            36|   662765|
|            38|   542079|
+--------------+---------+
only showing top 3 rows



As we know well from our analysis before **`Violation code- 21` occurs 768087 times**, **`Violation code- 36` occurs 662765 times** while **`Violation code- 38` occurs 542079 times** in our dataset contributing the highest percentage of tickets than other violation codes.

#### > Then, visit the website: http://www1.nyc.gov/site/finance/vehicles/services-violation-codes.page. It lists the fines associated with different violation codes. They’re divided into two categories: one for the highest-density locations in the city and the other for the rest of the city. For the sake of simplicity, take the average of the two.

Looking at the table given in the link, 

| CODE | DEFINITION | Manhattan 96th St. & below | All Other Areas | 
| ------ | ------ | ------ | ------ | 
| 21 | Street Cleaning: No parking where parking is not allowed by sign,<br /> street marking or traffic control device. | `$65` | `$45` | 
| 36 | Exceeding the posted speed limit in or near a designated school zone. | `$50` | `$50` |
| 38 | Failing to show a receipt or tag in the windshield.<br /> Drivers get a 5-minute grace period past the expired time on parking meter receipts. | `$65` | `$35` | 

The Average for the top three Violation codes would be as below,         

| Violation Code | Average Fine | 
| ------ | ------ | 
| 21 | `$55` |
| 36 | `$50` | 
| 38 | `$50` | 

#### > Using this information, find the total amount collected for the three violation codes with the maximum tickets. State the code that has the highest total collection.

In [53]:
# checking total collection under violation code 21
spark.sql("select `Violation Code`, count(*)*55 as Total_Collection from \
            (select * from nycparktbl where `Violation Code` = 21) \
            group by `Violation Code` ").show()

+--------------+----------------+
|Violation Code|Total_Collection|
+--------------+----------------+
|            21|        42244785|
+--------------+----------------+



In [54]:
# checking total collection under violation code 36
spark.sql("select `Violation Code`, count(*)*55 as Total_Collection from \
            (select * from nycparktbl where `Violation Code` = 36) \
            group by `Violation Code` ").show()

+--------------+----------------+
|Violation Code|Total_Collection|
+--------------+----------------+
|            36|        36452075|
+--------------+----------------+



In [55]:
# checking total collection under violation code 38
spark.sql("select `Violation Code`, count(*)*55 as Total_Collection from \
            (select * from nycparktbl where `Violation Code` = 38) \
            group by `Violation Code` ").show()

+--------------+----------------+
|Violation Code|Total_Collection|
+--------------+----------------+
|            38|        29814345|
+--------------+----------------+



Hence the total collection from the top three violation codes is in accordance with their sequence in terms of frequency, **Violation code `21` being the highest total collector with an amount of over 42 Million dollars** in the entire year 2017.

#### > What can you intuitively infer from these findings? 

From the above findings it is quite clear that, 
* There is a significant population in the city who tend to park their vehicle at the no parking areas. 
* Other than unauthorised parkings, Speed limit violations and Parking tag manifestation are also very common.

In [56]:
# stopping the spark
spark.stop()